In [57]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from helpers import *
from implementations import *
from crossvalidation import *
from preprocessing import *
from plots import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


IMPORTING TRAINING DATA

In [58]:
filename = 'train.csv'
data_folder = './data/'
file_path = data_folder + filename
y,tx,ids,features = load_train_data(file_path)

DEALING WITH CATEGORICAL VARIABLES

In [59]:
tx_train = categorical_values(tx,22,4)

IDENTIFYING AND DELETING THE COLUMNS HAVING MISSING VALUES

In [60]:
tx_train=managing_missing_values(tx_train)

HANDLING OUTLIERS - ??????

In [61]:
tx_train=capping_outliers(tx_train)

STANDARDIZING FEATURE MATRIX

In [62]:
tx_train,mean,std = standardize(tx_train)

ADD OFFSET (do not run this cell that if you want to expand later in the code)

In [21]:
y, tx_train = build_model_data(y, tx_train)

CROSS VALIDATION FOR HYPERPARAMETERS

In [52]:
indices = np.random.permutation(len(y))[:50000]

In [63]:
best_degree,best_lambda,best_err = cross_validation_demo_ridge(y[indices],tx_train[indices,:]
                                                   , 4, np.logspace(-5,-1,20),[1,2])

The choice of lambda which leads to the best test rmse is 0.00018 with a test rmse of 0.380. The best degree is 2.0


In [54]:
best_degree,best_lambda,best_err = cross_validation_demo_log(y[indices],tx_train[indices,:]
                                                   , 4, np.logspace(-5,-1,10),0.3,300,[1,2])

The choice of lambda which leads to the best test rmse is 0.00001 with a test rmse of 0.417. The best degree is 2.0


TEST MODEL ACCURACY

In [69]:
# Remember to set predict to 0, not -1
compute_accuracy(y,tx_train,divide_ratio = 0.7,lambda_ = best_lambda, deg = best_degree,method = 'logistic')

Average train accuracy: 0.8119987301587301
std train accuracy: 0.0005394531725475682
Average test accuracy: 0.8120222222222222
std train accuracy: 0.0012616900286500858


In [126]:
tx_train_temp = tx_train[:,1:-4]
phi = build_poly(tx_train_temp, degree=2)
tx_train = np.hstack((phi,tx_train[:,-4:]))

In [127]:
final_w,err = least_squares(y,tx_train)

GETTING CSV

In [130]:
test_data = 'test.csv'
tx_test,ids_test = load_test_data(data_folder+test_data)

HANDLING CATEGORICAL VARIABLE

In [131]:
tx_test = categorical_values(tx_test,22,4)

HANDLING MISSING DATA

In [132]:
tx_test=managing_missing_values(tx_test)

STANDARDIZING THE DATA

In [133]:
tx_test[:,:-4] = (tx_test[:,:-4]-mean)/std
_,tx_test= build_model_data(np.zeros(tx_test.shape[0]),tx_test)

In [134]:
tx_test_temp = tx_test[:,1:-4]
phi_test = build_poly(tx_test_temp, degree=2)
tx_test = np.hstack((phi_test,tx_test[:,-4:]))

PREDICTIONS

In [136]:
predictions = tx_test.dot(final_w)
predictions[predictions>=0.5]=1
predictions[predictions<0.5]=-1

CREATE SUBMISSION

In [137]:
create_submission(ids_test,predictions.astype(int),name = ['Id','Prediction'],file_name='./output/output_leastsquare.csv')